In [1]:
import numpy as np


In [2]:
dataset_filename = "affinity_dataset.txt"

In [3]:
X = np.loadtxt(dataset_filename)


In [5]:
n_samples, n_features = X.shape
print("This dataset has {0} samples and {1} features".format(n_samples, n_features))

This dataset has 100 samples and 5 features


**该数据集有100个记录、5个特征（变量）**

In [6]:
print(X[:5])

[[ 0.  0.  1.  1.  1.]
 [ 1.  1.  0.  1.  0.]
 [ 1.  0.  1.  1.  0.]
 [ 0.  0.  1.  1.  1.]
 [ 0.  1.  0.  0.  1.]]


In [7]:
# The names of the features, for your reference.
features = ["bread", "milk", "cheese", "apples", "bananas"]

**看一看怎么求“如果顾客购买了苹果，那么他也会购买香蕉”的支持度和置信度**

In [8]:
# 首先计算有多少顾客买了苹果
num_apple_purchases = 0
for sample in X:
    if sample[3] == 1:  # 判断是否买了苹果
        num_apple_purchases += 1
print("{0} people bought Apples".format(num_apple_purchases))

36 people bought Apples


In [9]:
# 有多少个即买了香蕉有买了苹果
# 计算上诉规则的个数
rule_valid = 0
rule_invalid = 0
for sample in X:
    if sample[3] == 1:  # 买了苹果
        if sample[4] == 1:
            # 也买了香蕉
            rule_valid += 1  # 符合规则+1
        else:
            # 买了苹果，但是没买香蕉
            rule_invalid += 1
print("{0} cases of the rule being valid were discovered".format(rule_valid))
print("{0} cases of the rule being invalid were discovered".format(rule_invalid))

21 cases of the rule being valid were discovered
15 cases of the rule being invalid were discovered


In [10]:
# 计算支持度和置信度
support = rule_valid  # 支持度就是规则发现的次数
confidence = rule_valid / num_apple_purchases  # 支持度除以买了苹果的记录数
print("The support is {0} and the confidence is {1:.3f}.".format(support, confidence))
# Confidence can be thought of as a percentage using the following:
print("As a percentage, that is {0:.1f}%.".format(100 * confidence))

The support is 21 and the confidence is 0.583.
As a percentage, that is 58.3%.


In [11]:
from collections import defaultdict
# Now compute for all possible rules
valid_rules = defaultdict(int)
invalid_rules = defaultdict(int)
num_occurences = defaultdict(int)

for sample in X:
    for premise in range(n_features):
        if sample[premise] == 0: continue
        # Record that the premise was bought in another transaction
        num_occurences[premise] += 1
        for conclusion in range(n_features):
            if premise == conclusion:  # 如果顾客买了苹果，然后又买了苹果，这样的规则是没有用的
                continue
            if sample[conclusion] == 1:
                # This person also bought the conclusion item
                valid_rules[(premise, conclusion)] += 1
            else:
                # This person bought the premise, but not the conclusion
                invalid_rules[(premise, conclusion)] += 1
support = valid_rules
confidence = defaultdict(float)
for premise, conclusion in valid_rules.keys():
    confidence[(premise, conclusion)] = valid_rules[(premise, conclusion)] / num_occurences[premise]

In [12]:
support

defaultdict(int,
            {(0, 1): 14,
             (0, 2): 4,
             (0, 3): 5,
             (0, 4): 17,
             (1, 0): 14,
             (1, 2): 7,
             (1, 3): 9,
             (1, 4): 19,
             (2, 0): 4,
             (2, 1): 7,
             (2, 3): 25,
             (2, 4): 27,
             (3, 0): 5,
             (3, 1): 9,
             (3, 2): 25,
             (3, 4): 21,
             (4, 0): 17,
             (4, 1): 19,
             (4, 2): 27,
             (4, 3): 21})

In [13]:
confidence

defaultdict(float,
            {(0, 1): 0.5185185185185185,
             (0, 2): 0.14814814814814814,
             (0, 3): 0.18518518518518517,
             (0, 4): 0.6296296296296297,
             (1, 0): 0.30434782608695654,
             (1, 2): 0.15217391304347827,
             (1, 3): 0.1956521739130435,
             (1, 4): 0.41304347826086957,
             (2, 0): 0.0975609756097561,
             (2, 1): 0.17073170731707318,
             (2, 3): 0.6097560975609756,
             (2, 4): 0.6585365853658537,
             (3, 0): 0.1388888888888889,
             (3, 1): 0.25,
             (3, 2): 0.6944444444444444,
             (3, 4): 0.5833333333333334,
             (4, 0): 0.288135593220339,
             (4, 1): 0.3220338983050847,
             (4, 2): 0.4576271186440678,
             (4, 3): 0.3559322033898305})

In [14]:
for premise, conclusion in confidence:
    premise_name = features[premise]
    conclusion_name = features[conclusion]
    print("Rule: If a person buys {0} they will also buy {1}".format(premise_name, conclusion_name))
    print(" - Confidence: {0:.3f}".format(confidence[(premise, conclusion)]))
    print(" - Support: {0}".format(support[(premise, conclusion)]))
    print("")

Rule: If a person buys cheese they will also buy apples
 - Confidence: 0.610
 - Support: 25

Rule: If a person buys cheese they will also buy bananas
 - Confidence: 0.659
 - Support: 27

Rule: If a person buys apples they will also buy cheese
 - Confidence: 0.694
 - Support: 25

Rule: If a person buys apples they will also buy bananas
 - Confidence: 0.583
 - Support: 21

Rule: If a person buys bananas they will also buy cheese
 - Confidence: 0.458
 - Support: 27

Rule: If a person buys bananas they will also buy apples
 - Confidence: 0.356
 - Support: 21

Rule: If a person buys bread they will also buy milk
 - Confidence: 0.519
 - Support: 14

Rule: If a person buys bread they will also buy apples
 - Confidence: 0.185
 - Support: 5

Rule: If a person buys milk they will also buy bread
 - Confidence: 0.304
 - Support: 14

Rule: If a person buys milk they will also buy apples
 - Confidence: 0.196
 - Support: 9

Rule: If a person buys apples they will also buy bread
 - Confidence: 0.139
 

In [15]:
def print_rule(premise, conclusion, support, confidence, features):
    premise_name = features[premise]
    conclusion_name = features[conclusion]
    print("Rule: If a person buys {0} they will also buy {1}".format(premise_name, conclusion_name))
    print(" - Confidence: {0:.3f}".format(confidence[(premise, conclusion)]))
    print(" - Support: {0}".format(support[(premise, conclusion)]))
    print("")

In [16]:
premise = 1
conclusion = 3
print_rule(premise, conclusion, support, confidence, features)

Rule: If a person buys milk they will also buy apples
 - Confidence: 0.196
 - Support: 9



### 1.3.5 排序找出最佳规则

In [17]:
# 按照支持度进行排序
from pprint import pprint # 为了打印美观
pprint(list(support.items()))  # 返回包含所有字典元素的列表

[((2, 3), 25),
 ((2, 4), 27),
 ((3, 2), 25),
 ((3, 4), 21),
 ((4, 2), 27),
 ((4, 3), 21),
 ((0, 1), 14),
 ((0, 3), 5),
 ((1, 0), 14),
 ((1, 3), 9),
 ((3, 0), 5),
 ((3, 1), 9),
 ((0, 2), 4),
 ((2, 0), 4),
 ((1, 4), 19),
 ((4, 1), 19),
 ((0, 4), 17),
 ((4, 0), 17),
 ((1, 2), 7),
 ((2, 1), 7)]


In [18]:
from operator import itemgetter   # 使用itemgetter类进行排序
sorted_support = sorted(support.items(), key=itemgetter(1), reverse=True)

In [19]:
for index in range(5):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_support[index][0]
    print_rule(premise, conclusion, support, confidence, features)

Rule #1
Rule: If a person buys cheese they will also buy bananas
 - Confidence: 0.659
 - Support: 27

Rule #2
Rule: If a person buys bananas they will also buy cheese
 - Confidence: 0.458
 - Support: 27

Rule #3
Rule: If a person buys cheese they will also buy apples
 - Confidence: 0.610
 - Support: 25

Rule #4
Rule: If a person buys apples they will also buy cheese
 - Confidence: 0.694
 - Support: 25

Rule #5
Rule: If a person buys apples they will also buy bananas
 - Confidence: 0.583
 - Support: 21



In [20]:
sorted_confidence = sorted(confidence.items(), key=itemgetter(1), reverse=True)

In [21]:
for index in range(5):
    print("Rule #{0}".format(index + 1))
    (premise, conclusion) = sorted_confidence[index][0]
    print_rule(premise, conclusion, support, confidence, features)

Rule #1
Rule: If a person buys apples they will also buy cheese
 - Confidence: 0.694
 - Support: 25

Rule #2
Rule: If a person buys cheese they will also buy bananas
 - Confidence: 0.659
 - Support: 27

Rule #3
Rule: If a person buys bread they will also buy bananas
 - Confidence: 0.630
 - Support: 17

Rule #4
Rule: If a person buys cheese they will also buy apples
 - Confidence: 0.610
 - Support: 25

Rule #5
Rule: If a person buys apples they will also buy bananas
 - Confidence: 0.583
 - Support: 21

